# Application Interface

Now that we have a trained model and a reasonable accuracy, we are going to build a programmer friendly app that can help the end user understand if the sentence correction is needed or not. Before going through this app, we request you to kindly make the note of following things -

1. This is a rudimentary application - no fancy UIs 
2. It just teases out how the prediction is being done for various sentences

In [2]:
!pip install colabcode
!pip install fastapi
!pip install pedantic

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# We will utilize pydantic to just define the input strings and we will also get Word net lemmatizer to lemmatize the text corpus

from pydantic import BaseModel
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize 

class Text_Clf(BaseModel):
    Sentence: str  
    class Config:
        schema_extra = {
            "example": {
                "Sentence": "There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â '' 11 , and that her niece and nephew , Anna and James Edward Austen , made further additions as late as 1814 ."
            }
        }
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

# Lemmatize the stop words
tokenizer=LemmaTokenizer()

In [5]:
#Import the pre trained vectorizer and model
import pickle
vectorizer = pickle.load(open("/content/drive/Shareddrives/MADS_Milestone2_Drive/App/vectorizer.pickle", "rb"))
model = pickle.load(open("/content/drive/Shareddrives/MADS_Milestone2_Drive/App/lr_trained.pickle", "rb"))

In [13]:
Test = vectorizer.transform(['There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â '' 11 , and that her niece and nephew , Anna and James Edward Austen , made further additions as late as 1814 .'])

str(model.predict(Test)[0])

'1'

In [8]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from fastapi import FastAPI
import pickle
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression

app = FastAPI()

@app.on_event("startup")
def load_model(token= LemmaTokenizer()):
    global vectorizer
    global model
    vectorizer = pickle.load(open("/content/drive/Shareddrives/MADS_Milestone2_Drive/App/vectorizer.pickle", "rb"))
    model = pickle.load(open("/content/drive/Shareddrives/MADS_Milestone2_Drive/App/lr_trained.pickle", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API'}

@app.post('/predict')
def get_text_classification(data: Text_Clf):
    received = data.dict()
    Sentence = list(received['Sentence'])
    pred_name = vectorizer.transform(Sentence)
    pred_val = list(model.predict(pred_name))[0]
    predict = str(pred_val)
    probability = model.predict_proba(pred_name)[:, 1]
    print(probability)
    return {'Prediction':str(model.predict(pred_name)[0])}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [11]:
# ENable a port connection
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [12]:
# Run this statement. You will see some thing like this for instance : 
# Public URL: NgrokTunnel: "https://cfd8-34-125-0-34.ngrok.io" -> "http://localhost:10000"
# Kindly copy the NgrokTunnel URL and place /docs at the very end
# In this example, just paste "https://cfd8-34-125-0-34.ngrok.io/docs"
# to see the model run
# For detailed readme, kindly refere to the 'App' folder
server.run_app(app=app)

Public URL: NgrokTunnel: "https://ef5a-35-227-111-193.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [65]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     49.206.39.3:0 - "GET / HTTP/1.1" 200 OK
INFO:     49.206.39.3:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     49.206.39.3:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     49.206.39.3:0 - "GET /openapi.json HTTP/1.1" 200 OK
[0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745 0.20691745
 0.20691745 0.20691745 0.2069

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [65]


In [ ]:
!pip install watermark

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.1
    Uninstalling importlib-metadata-4.8.1:
      Successfully uninstalled importlib-metadata-4.8.1


In [ ]:
%reload_ext watermark
%watermark
%watermark --iversions

Last updated: 2021-09-26T09:28:05.402323+00:00

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 5.5.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

nltk   : 3.2.5
IPython: 5.5.0
google : 2.0.3

